In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
def create_convnext_like_model(input_dim, num_classes, sliding_window_size=3):
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Reshape input to be compatible with Conv1D using Lambda layer
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)  # Shape: (batch_size, input_dim, 1)
    x = reshaped_input
    
    # Simulate ConvNeXt block using Conv1D layers
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 8)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    
    # Flatten the output of the final Conv1D layer
    x = Flatten()(x)
    
    # Global pooling and classification head
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        model = create_convnext_like_model(train_X.shape[1], test_y.shape[1])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=100, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [ ]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0014
test_accuracy: 0.9995
balanced_accuracy: 0.8570
roc_auc: 1.0000
pr_auc: 1.0000
f2: 0.9994
mcc: 0.9985
0_precision: 1.0000
0_recall: 0.9997
0_f1-score: 0.9998
0_support: 9117.0000
1_precision: 1.0000
1_recall: 1.0000
1_f1-score: 1.0000
1_support: 10.0000
2_precision: 0.9189
2_recall: 1.0000
2_f1-score: 0.9577
2_support: 34.0000
3_precision: 0.9994
3_recall: 0.9994
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 0.9969
4_recall: 1.0000
4_f1-score: 0.9985
4_support: 653.0000
5_precision: 0.0000
5_recall: 0.0000
5_f1-score: 0.0000
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9995
macro avg_precision: 0.8450
macro avg_recall: 0.8570
macro avg_f1-score: 0.8508
macro avg_support: 11600.0000
weighted avg_precision: 0.9993
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.9994
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/10 started...

Aggregated Metrics for 1 runs:
test_loss: 0.2629
test_accuracy: 0.8932
balanced_accuracy: 0.8076
roc_auc: 0.9903
pr_auc: 0.9604
f2: 0.8928
mcc: 0.8278
0_precision: 0.9092
0_recall: 0.8692
0_f1-score: 0.8887
0_support: 42368.0000
1_precision: 0.8919
1_recall: 0.9324
1_f1-score: 0.9117
1_support: 56661.0000
2_precision: 0.8615
2_recall: 0.8876
2_f1-score: 0.8744
2_support: 7151.0000
3_precision: 0.8530
3_recall: 0.6976
3_f1-score: 0.7675
3_support: 549.0000
4_precision: 0.8388
4_recall: 0.6440
4_f1-score: 0.7286
4_support: 1899.0000
5_precision: 0.7699
5_recall: 0.7495
5_f1-score: 0.7596
5_support: 3473.0000
6_precision: 0.9406
6_recall: 0.8730
6_f1-score: 0.9056
6_support: 4102.0000
accuracy: 0.8932
macro avg_precision: 0.8664
macro avg_recall: 0.8076
macro avg_f1-score: 0.8337
macro avg_support: 116203.0000
weighted avg_precision: 0.8933
weighted avg_recall: 0.8932
weighted avg_f1-score: 0.8926
weighted avg_support: 116203.0000
Run 2/10 started...

Aggregated Metri

In [ ]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_train.csv", "kdd_test.csv", is_string_labels = True, label_mapping=labels_map)

Memory usage of dataframe is 789.51 MB
Memory usage after optimization is: 196.60 MB
Decreased by 75.1%
Memory usage of dataframe is 197.38 MB
Memory usage after optimization is: 49.15 MB
Decreased by 75.1%
Verbose output only for first run...

Run 1/10 started...
Epoch 1/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 3093s 30s/step - accuracy: 0.9118 - loss: 0.3410
Epoch 2/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 3280s 32s/step - accuracy: 0.9963 - loss: 0.0145
Epoch 3/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 2931s 29s/step - accuracy: 0.9973 - loss: 0.0136
Epoch 4/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 3198s 32s/step - accuracy: 0.9985 - loss: 0.0069
Epoch 5/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 3155s 31s/step - accuracy: 0.9986 - loss: 0.0055
Epoch 6/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 3040s 30s/step - accuracy: 0.9988 - loss: 0.0050
Epoch 7/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 2884s 28s/step - accuracy: 0.9989 - loss: 0.0045
Epoch 8/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 2944s 29s/step - accuracy: 0.9991 - loss: 0.0037
Epoch 9/100
101

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/9 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0028
test_accuracy: 0.9992
balanced_accuracy: 0.9794
roc_auc: 1.0000
pr_auc: 0.9999
f2: 0.9992
mcc: 0.9970
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 1.0000
1_recall: 0.9981
1_f1-score: 0.9991
1_support: 2657.0000
2_precision: 0.9934
2_recall: 0.9978
2_f1-score: 0.9956
2_support: 908.0000
3_precision: 0.9943
3_recall: 0.9943
3_f1-score: 0.9943
3_support: 522.0000
4_precision: 0.9863
4_recall: 0.9863
4_f1-score: 0.9863
4_support: 293.0000
5_precision: 0.9853
5_recall: 0.9963
5_f1-score: 0.9908
5_support: 269.0000
6_precision: 0.9826
6_recall: 0.9741
6_f1-score: 0.9784
6_support: 116.0000
7_precision: 0.9787
7_recall: 0.8679
7_f1-score: 0.9200
7_support: 53.0000
8_precision: 0.9778
8_recall: 1.0000
8_f1-score: 0.9888
8_support: 44.0000
accuracy: 0.9992
macro avg_precision: 0.9887
macro avg_recall: 0.9794
macro avg_f1-score: 0.9837
macro avg_support: 31724.0000
weighted avg_prec